**Import các thư viện cần thiết**

In [1]:
import math
import numpy as np
from scipy.spatial import distance

**Tập dữ liệu văn bản/tài liệu mẫu và truy vấn**

In [2]:
# Tập văn bản/tài liệu (D)
docs = [
    'data mining is awesome',
    'information retrieval is awesome',
    'natural language processing is awesome'
];

doc_len = len(docs)

# Truy vấn (q)
query = 'information retrieval'

# Để cho tiện lợi trong việc xử lý chúng ta sẽ gán câu truy vấn là 1 tài liệu cuối cùng
docs.append(query)

**Đọc từng tài liệu/văn bản và thực hiện tách từ - gán id cho các token và xây dựng tập từ vựng**

In [3]:
# Cấu trúc dữ liệu dạng dictionary -> <key: id, value: token>
id_token_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: token, value: id>
token_id_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: doc_id, value: [id_token_xuất_hiện_1, id_token_xuất_hiện_2, v.v. ]>
doc_id_token_ids_dict = {}

# Gán id của token đầu tiên là [0]
token_id = 0

# Duyệt qua từng tài liệu/văn bản có trong mảng
for doc_idx, doc in enumerate(docs):
  # Tách các từ riêng biệt trong tài liệu/văn bản bằng khoảng trắng " "
  tokens = doc.split(' ')
  
  # Khởi tạo danh sách các token_ids xuất hiện trong tài liệu/văn bản
  doc_token_ids = []

  # Đọc qua từng token trong mỗi tài liệu/văn bản
  for token in tokens:
    # Kiểm tra chiều dài từ khóa > 0
    if len(token) > 0:
      if token not in token_id_dict.keys():
        token_id_dict[token] = token_id
        id_token_dict[token_id] = token
        # Tăng id của token tiếp theo lên 1
        token_id+=1

      doc_token_ids.append(token_id_dict[token])
  doc_id_token_ids_dict[doc_idx] = doc_token_ids

# Xuất ra màn hình kích thước tập từ vựng (vocabulary)
vocab_size = len(id_token_dict.keys())
print('Kích thước tập từ vựng: [{}]'.format(vocab_size))
print('Tập từ vựng (V):')
print(id_token_dict)

Kích thước tập từ vựng: [9]
Tập từ vựng (V):
{0: 'data', 1: 'mining', 2: 'is', 3: 'awesome', 4: 'information', 5: 'retrieval', 6: 'natural', 7: 'language', 8: 'processing'}


**Chuyển đổi các tài liệu/văn bản và câu truy vấn về dạng  encoded vector**

In [4]:
# Cấu trúc dữ liệu dạng dictionary -> <key: doc_id, value: encoded_vector>
doc_id_encoded_vector_dict = {}

# Duyệt qua từng tài liệu/văn bản có trong mảng
for doc_idx in doc_id_token_ids_dict.keys():
  # Khởi tạo một vector có số chiều = vocab_size và các giá trị [0] -> [0, 0, 0, ...]
  encoded_vector = np.zeros(vocab_size)

  # Danh sách các token_ids xuất hiện trong tài liệu/văn bản
  doc_token_ids = doc_id_token_ids_dict[doc_idx]
  
  # Duyệt qua từng token xuất hiện trong tài liệu/văn bản
  for token_id in doc_token_ids:
    # Gán vị trí - cột/chiều thứ (i) của vector giá trị 1 - tương đương với vị trí xuất hiện của token
    encoded_vector[token_id] = 1
  doc_id_encoded_vector_dict[doc_idx] = encoded_vector

#  encode của truy vấn (q) là tài liệu cuối cùng
query_encoded_vector = doc_id_encoded_vector_dict[doc_len]

# Xóa query đã được mã hóa thành dạng  vector ra khỏi danh sách
del doc_id_encoded_vector_dict[doc_len]

print('Truy vấn được chuyển đổi về dạng  vector:')
print(query_encoded_vector)

Truy vấn được chuyển đổi về dạng  vector:
[0. 0. 0. 0. 1. 1. 0. 0. 0.]


**Tìm danh sách các tài liệu liên quan đến truy vấn (Tích vô hướng)**

**Để xác định tích vô hướng của 2 vectors ($\vec{a}$) và $\vec{b})$ cùng có số chiều là ($T$), hay: $\vec{a}·\vec{b}$, ta làm như sau:**
$$\vec{a}·\vec{b}=\sum_{i=1}^{T}a_{i}*b_{i} = (a_{1}*b_{1})+(a_{2}*b_{2})+...+(a_{T}*b_{T})$$

In [5]:
# Viết hàm tính tính vô hướng của 2 vectors
def dot_product(a, b):
  result = 0;
  for (ai, bi) in zip(a, b):
    result += ai * bi
  return result

In [6]:
# Duyệt qua danh sách các tài liệu/văn bản (đã mã hóa ở dạng  vectors)
for doc_idx in doc_id_encoded_vector_dict.keys():
  doc_encoded_vector = doc_id_encoded_vector_dict[doc_idx]
  
  # Tính tích vô hướng giữa hai vectors tài liệu và truy vấn
  dot_product_sim = dot_product(query_encoded_vector, doc_encoded_vector)
  
  # Hoặc dùng thư viện numpy -> numpy.dot
  #dot_product_sim = np.dot(query_encoded_vector, doc_encoded_vector)
  
  print('Tài liệu: [{}], tương đồng (dot product): [{}]'.format(doc_idx, dot_product_sim))

Tài liệu: [0], tương đồng (dot product): [0.0]
Tài liệu: [1], tương đồng (dot product): [2.0]
Tài liệu: [2], tương đồng (dot product): [0.0]


**Tìm danh sách các tài liệu liên quan đến truy vấn (Khoảng cách Euclid)**

**Để xác định khoảng cách Euclid của 2 vectors ($\vec{a}$) và $\vec{b})$, ký hiệu: $ED(\vec{a},\vec{b})$ ta làm như sau:**

$$ED(\vec{a},\vec{b})=\sqrt{\sum_{i=1}^{T}(a_{i}-b_{i})^{2}} = \sqrt{(a_{1}-b_{1})^{2}+(a_{2}-b_{2})^{2}+...+(a_{T}-b_{T})^{2}}$$

In [7]:
# Viết hàm tính khoảng cách Euclid của 2 vectors
def euclid_distance(a, b):
  result = 0;
  for (ai, bi) in zip(a, b):
    result += (ai - bi)**2
  return math.sqrt(result)

In [8]:
# Duyệt qua danh sách các tài liệu/văn bản (đã mã hóa ở dạng  vectors)
for doc_idx in doc_id_encoded_vector_dict.keys():
  doc_encoded_vector = doc_id_encoded_vector_dict[doc_idx]
  
  # Tính khoảng cách Euclid giữa hai vectors tài liệu và truy vấn
  ed = euclid_distance(query_encoded_vector, doc_encoded_vector)
  
  # Hoặc dùng thư viện scipy distance -> distance.euclidean
  #ed = distance.euclidean(query_encoded_vector, doc_encoded_vector)

  print('Tài liệu: [{}], tương đồng (khoảng cách Euclid): [{:.6f}]'.format(doc_idx, ed))

Tài liệu: [0], tương đồng (khoảng cách Euclid): [2.449490]
Tài liệu: [1], tương đồng (khoảng cách Euclid): [1.414214]
Tài liệu: [2], tương đồng (khoảng cách Euclid): [2.645751]


**Tìm danh sách các tài liệu liên quan đến truy vấn (Tương đồng cosine)**
**Để xác định khoảng cách Euclid của 2 vectors ($\vec{a}$) và $\vec{b})$, cùng có số chiều là ($T$), ký hiệu: $CS(\vec{a},\vec{b})$ ta làm như sau:**

$$CS(\vec{a},\vec{b})=\frac{\vec{a}·\vec{b}}{\|\vec{a}\|\|\vec{b}\|}=\frac{\sum_{i=1}^{T}a_{i}*b_{i}}{\sqrt{\sum_{i=1}^{T}a_{i}^{2}}\sqrt{\sum_{i=1}^{T}b_{i}^{2}}}$$

In [9]:
# Viết hàm tính tương đồng cosine giữa hai vectors
def cosine_sim(a, b):
  dot_prod = 0
  magnitude_a = 0
  magnitude_b = 0
  for (ai, bi) in zip(a, b):
    dot_prod += ai * bi
    magnitude_a += ai ** 2
    magnitude_b += bi ** 2
  return dot_prod / (math.sqrt(magnitude_a) * math.sqrt(magnitude_b))

# Hoặc dùng thư viện numpy với hàm dot và norm
def cosine_sim_v2(a, b):
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [10]:
# Duyệt qua danh sách các tài liệu/văn bản (đã mã hóa ở dạng  vectors)
for doc_idx in doc_id_encoded_vector_dict.keys():
  doc_encoded_vector = doc_id_encoded_vector_dict[doc_idx]
  
  # Tính tương đồ cosine giữa hai vectors tài liệu và truy vấn
  cs = cosine_sim(query_encoded_vector, doc_encoded_vector)
  
  # Hoặc dùng thư viện scipy distance -> distance.cosine
  # Vì là khoảng cách cosine nên để tính tương đồng cosine chúng ta sẽ lấy 1 - khoảng cách
  #cs = 1 - distance.cosine(query_encoded_vector, doc_encoded_vector)

  print('Tài liệu: [{}], tương đồng (Tương đồng cosine): [{:.6f}]'.format(doc_idx, cs))

Tài liệu: [0], tương đồng (Tương đồng cosine): [0.000000]
Tài liệu: [1], tương đồng (Tương đồng cosine): [0.707107]
Tài liệu: [2], tương đồng (Tương đồng cosine): [0.000000]
